In [7]:
import dotenv
dotenv.load_dotenv()

True

In [10]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(api_wrapper=TavilySearchAPIWrapper())


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())



available_tools = [tavily_tool, wikipedia]




In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [12]:

from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from typing import List


class Question(BaseModel):
    ask_to: str = Field(description="Expert To Ask the Question To")
    question: str = Field(description="Question to ask the respective expert to get their opinion")

class QuestionsList(BaseModel):
    questions: List[Question] = Field(description="List of questions to ask to various experts")
questions_list_parser = PydanticOutputParser(pydantic_object=QuestionsList)

In [13]:
members = {
    "MarketExpert":"Market expert in london regarding rental and on-sale properties.", 
    "LocationExpert":"Location expert and strategist in london", 
    "FinanceExpert":"Financial Expert in London"
}

In [19]:
from langchain.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, AIMessage
prompt_to_head_string = (
    "You are a head of a real estate company. "
    "You've following co-workers in your company: {members}. "
    "User Asks the following: \n{query}\n"
    "Based on the user query, generate ten or more questions to ask one or more experts. "
    "Make sure you ask the experts the questions based on their expertise."
    "\n{format_instructions}"
)
prompt_to_head = PromptTemplate(
    template=prompt_to_head_string,
    input_variables=["query"],
    partial_variables={"members":members, "format_instructions":questions_list_parser.get_format_instructions()}
)
query_generation_chain = prompt_to_head | llm | questions_list_parser


In [20]:
from langchain.prompts import PromptTemplate, MessagesPlaceholder, ChatPromptTemplate
from langchain.agents import AgentExecutor, create_openai_tools_agent

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser



def create_expert(tools, role, role_description):

    sys_prompt_str = ("You're an expert of a real estate consultancy. "+
            "Your expertise is "+role+" (Description: "+role_description+"). ")
    
    prompt_str = (
        "You're asked the following by the head of the consultancy. \n"
        "{input}\n"
        "Based on the query, give a short and concise answer, two or three sentences in plain readable text. "
        "Use Tools if necessary. "
    )
    sprompt = ChatPromptTemplate.from_messages([
        (
            "system", sys_prompt_str
        ),
        ("user", prompt_str),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])
    llm_with_tools = llm.bind_tools(tools)
    agent = (
                {
                    "input": lambda x: x["input"],
                    "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                        x["intermediate_steps"]
                    ),
                }
                | sprompt
                | llm_with_tools
                | OpenAIToolsAgentOutputParser()
            )
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    return agent_executor


In [21]:
marketing_agent = create_expert(available_tools, "MarketExpert", members["MarketExpert"])

def marketing_agent_node(q):
    resp = marketing_agent.invoke(dict(input=q, tools=available_tools))
    return resp['output']



finance_agent = create_expert(available_tools, "FinanceExpert", members["FinanceExpert"])

def finance_agent_node(q):
    resp = finance_agent.invoke(dict(input=q, tools=available_tools))
    return resp['output']



location_agent = create_expert(available_tools, "LocationExpert", members["LocationExpert"])

def location_agent_node(q):
    resp = location_agent.invoke(dict(input=q, tools=available_tools))
    return resp['output']



agent_nodes_map = dict(
    MarketExpert=marketing_agent_node,
    FinanceExpert=finance_agent_node,
    LocationExpert=location_agent_node
)




In [36]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool



@tool
def filter_question_responses(q):
    """Filter the Given Question Parsed Output"""
    return q.questions


@tool
def ask_questions_to_expert(questions, done_list=[]):
    """Ask Questions to related experts"""
    if len(questions) == 0:
        return done_list
    print("i=", len(done_list))
    to_process_q = questions[0]
    done_list.append((
                to_process_q.ask_to,
                to_process_q.question,
                agent_nodes_map[to_process_q.ask_to](to_process_q.question)
            )
    )
    questions = questions[1:]   
    return ask_questions_to_expert.invoke(questions, done_list)

In [37]:
responses = query_generation_chain.invoke(dict(query="Is london a good business starting place?"))


In [39]:
qs = responses.questions

In [41]:
ask_questions_to_expert.invoke(dict(questions=qs))

i= 2


AttributeError: 'list' object has no attribute 'items'

In [ ]:

# def qa_mapping_to_prompt(q):
#     return "\n\n".join(["Question: "+i[1]+"\nAnswer: "+i[2] for i in q])